In [ ]:
# All baseline models are run for inference using this code
# Runs on both CPU/GPU, faster inference on GPU

# Please upload CSFCube-master.zip to the colab notebook
# This zip file is present in our code base

# **Install libraries and prepare CSF-Cube dataset**

In [ ]:
!pip install transformers
!pip install -U sentence-transformers
!pip install simcse
!pip install gdown
!pip install jsonlines
from sentence_transformers import SentenceTransformer
from simcse import SimCSE

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.7 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=4ff41906856ee332477339fc9df6e8d4f72a193e4f59bef98688b9c6fa9306ab
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers
Looking in indexes: https://pypi.

In [ ]:
!unzip CSFCube-master.zip

Archive:  CSFCube-master.zip
   creating: CSFCube-master/
  inflating: CSFCube-master/.gitignore  
  inflating: CSFCube-master/LICENSE.md  
  inflating: CSFCube-master/README.md  
  inflating: CSFCube-master/abstracts-csfcube-preds.jsonl  
  inflating: CSFCube-master/ann_guidelines.pdf  
  inflating: CSFCube-master/datasheet.md  
   creating: CSFCube-master/eval_scripts/
  inflating: CSFCube-master/eval_scripts/rank_metrics.py  
  inflating: CSFCube-master/eval_scripts/ranking_eval.py  
  inflating: CSFCube-master/eval_scripts/requirements.txt  
   creating: CSFCube-master/eval_scripts/sample_ranked_pools/
  inflating: CSFCube-master/eval_scripts/sample_ranked_pools/test-pid2pool-csfcube-specter-background-ranked.json  
  inflating: CSFCube-master/eval_scripts/sample_ranked_pools/test-pid2pool-csfcube-specter-method-ranked.json  
  inflating: CSFCube-master/eval_scripts/sample_ranked_pools/test-pid2pool-csfcube-specter-result-ranked.json  
  inflating: CSFCube-master/evaluation_splits.

In [ ]:
!gdown https://drive.google.com/uc?id=1THQvaypgyLgpOSHdHRMca7j1Rb44yLVe

Downloading...
From: https://drive.google.com/uc?id=1THQvaypgyLgpOSHdHRMca7j1Rb44yLVe
To: /content/specter_triplet_4.pt
100% 440M/440M [00:07<00:00, 56.4MB/s]


In [ ]:
!gdown https://drive.google.com/uc?id=1PG7_SfihyL1OHEPR32SbFISqY6wyu4xc

Downloading...
From: https://drive.google.com/uc?id=1PG7_SfihyL1OHEPR32SbFISqY6wyu4xc
To: /content/scincl_triplet_4.pt
100% 440M/440M [00:05<00:00, 78.8MB/s]


In [ ]:
!gdown https://drive.google.com/uc?id=1ICTJSb6uziPbmbI4J12RnONNOx07y_O8

Downloading...
From: https://drive.google.com/uc?id=1ICTJSb6uziPbmbI4J12RnONNOx07y_O8
To: /content/finetuned-sentbert-pp-parade-train.pt
100% 268M/268M [00:02<00:00, 119MB/s]


# **Parse CSF-Cube dataset into Python dictionaries**

In [ ]:
import json
def parse_dataset(file_data,file_background,file_method,file_result):
  data = dict()
  with open(file_data,'r') as f:
    abs_list = list(f)
  for paper in abs_list:
    paper = json.loads(paper)
    dict_paper = dict()
    dict_paper['title'] = paper['title']
    dict_paper['abstract'] = paper['abstract']
    dict_paper['pred_labels'] = paper['pred_labels']
    try:
      print(data[paper['paper_id']])
    except:
      data[paper['paper_id']] = dict_paper
  file_list = [file_background,file_method,file_result]
  result_list = []
  for f in file_list:
    with open(f,'r') as json_file:
      results_data = json.load(json_file)
    result_dict = dict()
    for k in results_data.keys():
      try:
        print(result_dict[k].keys())
      except:
        result_dict[k] = dict()
      result_dict[k]['cands'] = results_data[k]['cands']
      result_dict[k]['relevance_adju'] = results_data[k]['relevance_adju']
    result_list.append(result_dict)
  return data,result_list[0],result_list[1],result_list[2]

In [ ]:
sentences,bg_ranking,method_ranking,result_ranking = parse_dataset('/content/CSFCube-master/abstracts-csfcube-preds.jsonl','/content/CSFCube-master/test-pid2anns-csfcube-background.json','/content/CSFCube-master/test-pid2anns-csfcube-method.json','/content/CSFCube-master/test-pid2anns-csfcube-result.json')

# **Load pre-trained language models**

In [ ]:
from transformers import *
import torch
from numpy import dot
from numpy.linalg import norm

def init_model_tokenizer(i):
  if i==0:
    tokenizer = AutoTokenizer.from_pretrained('allenai/specter')
    model = AutoModel.from_pretrained('allenai/specter')
  if i==1:
    tokenizer = AutoTokenizer.from_pretrained('allenai/specter')
    model = AutoModel.from_pretrained('allenai/specter')
  if i==2:
    tokenizer = AutoTokenizer.from_pretrained('malteos/scincl')
    model = AutoModel.from_pretrained('malteos/scincl') 
  if i==3:
    tokenizer = AutoTokenizer.from_pretrained('malteos/scincl')
    model = AutoModel.from_pretrained('malteos/scincl')
  if i==4:
    tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
    model = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased')

  if i==10:
    model = SentenceTransformer('paraphrase-TinyBERT-L6-v2')
    tokenizer = None
  if i==11:
    model = SentenceTransformer('paraphrase-TinyBERT-L6-v2')
    tokenizer = None
  if i==12:
    model = SentenceTransformer('nli-roberta-base-v2')
    tokenizer = None
  if i==13:
    model = SimCSE("princeton-nlp/unsup-simcse-bert-base-uncased")
    tokenizer = None
  if i==14:
    model = SimCSE("princeton-nlp/sup-simcse-bert-base-uncased")
    tokenizer = None


  return model,tokenizer

def baseline_model(model_id, model, tokenizer, texts, max_length):
    if model_id == 10 or model_id == 11 or model_id == 12 or model_id == 13 or model_id == 14:
        if model_id == 10 or model_id == 11 or model_id == 12:
          device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
          model.to(device)
          model.eval()
        cls_embeddings = model.encode(texts)
        return cls_embeddings
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()
    with torch.no_grad():
        tokens = tokenizer(texts, max_length=max_length, truncation=True, padding=True, return_tensors='pt')
        tokens.to(device)
        outputs = model(**tokens)

        # Get the output embeddings
        embeddings = outputs.last_hidden_state
        cls_embedding = embeddings[:, 0, :]
        cls_embedding = cls_embedding.cpu().detach().numpy()
        return cls_embedding

In [ ]:
keys = [0, 1, 2, 3, 4, 10, 11, 12, 13, 14]
model_dict = dict()
tokenizer_dict = dict()

for k in keys:
  model_dict[k],tokenizer_dict[k] = init_model_tokenizer(k)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--allenai--specter/snapshots/5b0bc1dc47329dfaaa100a5c14cf8942b258591e/config.json
Model config BertConfig {
  "_name_or_path": "allenai/specter",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.27.4",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 31116
}



loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--allenai--specter/snapshots/5b0bc1dc47329dfaaa100a5c14cf8942b258591e/vocab.txt
loading file tokenizer.json from cache at None
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--allenai--specter/snapshots/5b0bc1dc47329dfaaa100a5c14cf8942b258591e/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--allenai--specter/snapshots/5b0bc1dc47329dfaaa100a5c14cf8942b258591e/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--allenai--specter/snapshots/5b0bc1dc47329dfaaa100a5c14cf8942b258591e/config.json
Model config BertConfig {
  "_name_or_path": "allenai/specter",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act":

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--allenai--specter/snapshots/5b0bc1dc47329dfaaa100a5c14cf8942b258591e/pytorch_model.bin
All model checkpoint weights were used when initializing BertModel.

All the weights of BertModel were initialized from the model checkpoint at allenai/specter.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertModel for predictions without further training.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--allenai--specter/snapshots/5b0bc1dc47329dfaaa100a5c14cf8942b258591e/config.json
Model config BertConfig {
  "_name_or_path": "allenai/specter",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 307

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--malteos--scincl/snapshots/5cab047e2007fcc749ca78c2728142c4df5df1c4/config.json
Model config BertConfig {
  "_name_or_path": "malteos/scincl",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.27.4",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 31090
}



loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--malteos--scincl/snapshots/5cab047e2007fcc749ca78c2728142c4df5df1c4/vocab.txt
loading file tokenizer.json from cache at None
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--malteos--scincl/snapshots/5cab047e2007fcc749ca78c2728142c4df5df1c4/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--malteos--scincl/snapshots/5cab047e2007fcc749ca78c2728142c4df5df1c4/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--malteos--scincl/snapshots/5cab047e2007fcc749ca78c2728142c4df5df1c4/config.json
Model config BertConfig {
  "_name_or_path": "malteos/scincl",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gel

...

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--princeton-nlp--unsup-simcse-bert-base-uncased/snapshots/218baf6bc48c463a53f5b4e1c568299537fab124/vocab.txt
loading file tokenizer.json from cache at None
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--princeton-nlp--unsup-simcse-bert-base-uncased/snapshots/218baf6bc48c463a53f5b4e1c568299537fab124/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--princeton-nlp--unsup-simcse-bert-base-uncased/snapshots/218baf6bc48c463a53f5b4e1c568299537fab124/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--princeton-nlp--unsup-simcse-bert-base-uncased/snapshots/218baf6bc48c463a53f5b4e1c568299537fab124/config.json
Model config BertConfig {
  "_name_or_path": "princeton-nlp/unsup-simcse-bert-base-uncased",
  "architectures

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--princeton-nlp--unsup-simcse-bert-base-uncased/snapshots/218baf6bc48c463a53f5b4e1c568299537fab124/pytorch_model.bin
All model checkpoint weights were used when initializing BertModel.

All the weights of BertModel were initialized from the model checkpoint at princeton-nlp/unsup-simcse-bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertModel for predictions without further training.


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--princeton-nlp--sup-simcse-bert-base-uncased/snapshots/2d82fab19ac3a73a20dd20333d27eb8a52d6e97f/config.json
Model config BertConfig {
  "_name_or_path": "princeton-nlp/sup-simcse-bert-base-uncased",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.27.4",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--princeton-nlp--sup-simcse-bert-base-uncased/snapshots/2d82fab19ac3a73a20dd20333d27eb8a52d6e97f/vocab.txt
loading file tokenizer.json from cache at None
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--princeton-nlp--sup-simcse-bert-base-uncased/snapshots/2d82fab19ac3a73a20dd20333d27eb8a52d6e97f/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--princeton-nlp--sup-simcse-bert-base-uncased/snapshots/2d82fab19ac3a73a20dd20333d27eb8a52d6e97f/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--princeton-nlp--sup-simcse-bert-base-uncased/snapshots/2d82fab19ac3a73a20dd20333d27eb8a52d6e97f/config.json
Model config BertConfig {
  "_name_or_path": "princeton-nlp/sup-simcse-bert-base-uncased",
  "architectures": [
    "

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--princeton-nlp--sup-simcse-bert-base-uncased/snapshots/2d82fab19ac3a73a20dd20333d27eb8a52d6e97f/pytorch_model.bin
All model checkpoint weights were used when initializing BertModel.

All the weights of BertModel were initialized from the model checkpoint at princeton-nlp/sup-simcse-bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertModel for predictions without further training.


# **Ranked retrieval for candidates**

In [ ]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

def L2_dist_ranking(cls_embeddings,query_id,cand_ids,results_dict):
  query_cls = cls_embeddings[0]
  if query_id in results_dict.keys():
    print('err')
    return
  results = []
  for i in range(1,len(cls_embeddings)):
    l2_norm = np.linalg.norm(query_cls - cls_embeddings[i])
    results.append([cand_ids[i-1],l2_norm])
  results = sorted(results, key = lambda x: x[1])
  
  for i in range(len(results)):
    results[i][1] = float(results[i][1])
  results_dict[query_id] = results
  return

def cosine_similarity(query_embeddings, cls_embeddings):
  max_cosine_sim = 0.0
  for embd_query in query_embeddings:
    for embd_cand in cls_embeddings:
      curr_cosine_sim = dot(embd_query, embd_cand)/(norm(embd_cand)*norm(embd_query))
      if curr_cosine_sim > max_cosine_sim:
        max_cosine_sim = curr_cosine_sim

  return max_cosine_sim

  

In [ ]:
import json

def write_results(results_dict,model_name,facet):
  file_name = "test-pid2pool-csfcube-" + model_name + "-" + facet + "-ranked.json"
  with open(file_name,'w') as f:
    json.dump(results_dict,f)


In [ ]:
import jsonlines
def getTitlesAndAbstracts(filename):
  with jsonlines.open(filename) as reader:
    papers = [paper for paper in reader]
  paper_map = {paper['paper_id']: {'title': paper['title'], 'abstract': paper['abstract']} for paper in papers}
  return paper_map

In [ ]:
papers = getTitlesAndAbstracts('/content/CSFCube-master/abstracts-csfcube-preds.jsonl')

In [ ]:
def getCandTitlesAndAbstract(ranking_dict, query_id, results, papers):
  candidate_titles = []
  candidate_abstracts = []
  candidate_scores = []

  cands = ranking_dict[query_id]['cands']
  relevance = ranking_dict[query_id]['relevance_adju']

  for i, result in enumerate(results):
    candidate_titles.append(papers[result[0]]['title'])
    abstract = ""
    for abs in papers[result[0]]['abstract']:
      abstract+=abs
    candidate_abstracts.append(abstract)
    candidate_scores.append(relevance[cands.index(result[0])])
    if i == 20:
      break
  query = papers[query_id]['title']
  return candidate_titles, candidate_abstracts, candidate_scores

In [ ]:
import json
import os

model_map = {'specter':0, 'finetuned-specter':1,'scincl':2, 'finetuned-scincl':3,'scibert_uncased':4,'sentbert-pp':10,'finetuned-sentbert-pp':11, 'sentbert-nli':12, 'unsimcse':13, 'susimcse': 14}
ranking_dictionary = {'background':bg_ranking,'method':method_ranking,'result':result_ranking}

def prepare_batch(query_id,data_dict,ranking_dict,query_facet,cand_facet):
  titles = []
  abstracts = []
  query_title = data_dict[query_id]['title']
  titles.append(query_title)
  query_abstract = []
  for i in range(len(data_dict[query_id]['abstract'])):
    curr_label = data_dict[query_id]['pred_labels'][i]
    if query_facet == "abstract" or query_facet == curr_label:
      query_abstract.append(data_dict[query_id]['abstract'][i])
    elif query_facet == "background_label" and curr_label=="objective_label":
      query_abstract.append(data_dict[query_id]['abstract'][i])
    elif query_facet == "objective_label" and curr_label=="background_label":
       query_abstract.append(data_dict[query_id]['abstract'][i])
  abstracts.append(query_abstract)

  for j in range(len(ranking_dict[query_id]['cands'])):
    cand_id = ranking_dict[query_id]['cands'][j]
    cand_title = data_dict[cand_id]['title']
    titles.append(cand_title)
    cand_abstract = []
    for i in range(len(data_dict[cand_id]['abstract'])):
      curr_label = data_dict[cand_id]['pred_labels'][i]
      if cand_facet == "abstract" or cand_facet == curr_label:
        cand_abstract.append(data_dict[cand_id]['abstract'][i])
      elif cand_facet == "background_label" and curr_label=="objective_label":
        cand_abstract.append(data_dict[cand_id]['abstract'][i])
      elif cand_facet == "objective_label" and curr_label=="background_label":
        cand_abstract.append(data_dict[cand_id]['abstract'][i])
    abstracts.append(cand_abstract)    

  return titles,abstracts

def eval_model(model_name,data_dict,query_facet,cand_facet,ranking_facet, papers):
  model_id = model_map[model_name]
  ranking_dict = ranking_dictionary[ranking_facet]
  for query in ranking_dict:
    print(query, papers[query]['title'])
  query_id = input("Enter a Query ID: ")
  query = papers[query_id]['title']
  model,tokenizer = model_dict[model_id],tokenizer_dict[model_id]
  results_dict = dict()
  batch_data = prepare_batch(query_id,data_dict,ranking_dict,query_facet,cand_facet)
  if model_name == 'specter' or model_name == 'finetuned-specter' or model_name == 'scincl' or model_name == 'finetuned-scincl' or model_name == 'scibert_uncased':      
    for i in range(len(batch_data[1])):
      abs = ""
      for j in batch_data[1][i]:
        abs += j
      batch_data[1][i] = abs
    if model_name == 'scibert_uncased':
      title_abs = batch_data[1]
    else:
      title_abs = [batch_data[0][i] + tokenizer.sep_token + batch_data[1][i] for i in range(len(batch_data[0]))]
    cls_embeddings = baseline_model(model_id,model,tokenizer,title_abs,512)
    L2_dist_ranking(cls_embeddings,query_id,ranking_dict[query_id]['cands'],results_dict)
  elif model_name == 'sentbert-pp' or model_name == 'finetuned-sentbert-pp' or model_name=='sentbert-nli' or model_name == 'unsimcse' or model_name == 'susimcse':
    abstracts = batch_data[1]
    results = []
    if(len(batch_data[1][0])==0):
      batch_data[1][0] = ['']
    query_embeddings = baseline_model(model_map[model_name], model, None, batch_data[1][0], None)
    for i in range(1, len(batch_data[1])):
      if len(batch_data[1][i])==0:
        batch_data[1][i] = ['']
      cls_embeddings = baseline_model(model_map[model_name], model, None, batch_data[1][i], None)
      max_cosine_sim = cosine_similarity(query_embeddings, cls_embeddings)
      results.append([ranking_dict[query_id]['cands'][i-1], max_cosine_sim])
    results = sorted(results, key = lambda x: x[1], reverse=True)
    for i in range(len(results)):
      results[i][1] = float(results[i][1])
    results_dict[query_id] = results
  
  return getCandTitlesAndAbstract(ranking_dict, query_id, results_dict[query_id], papers), query

In [ ]:
from IPython.display import HTML, display

def displayResults(candidate_titles, candidate_abstracts, candidate_scores, query):
  data = []
  print("\n\n\nQuery Title:",query)
  data.append(["Candidate Title", "Candidate Abstract", "Score"])

  for i in range(len(candidate_titles)):
      data.append([f"{candidate_titles[i]}:", candidate_abstracts[i], candidate_scores[i]])

  table_style = """
      <style>
      table {
          border-collapse: collapse;
          font-size: 16px;
          margin-top: 20px;
          margin-bottom: 20px;
          width: 100%;
          text-align: left;
      }
      th {
          font-weight: bold;
          background-color: #f2f2f2;
          border: 1px solid #ddd;
          padding: 10px;
          text-align: center;
      }
      td {
          border: 1px solid #ddd;
          padding: 10px;
      }
      .title {
          font-weight: bold;
          font-size: 18px;
          color: #333;
          margin-bottom: 5px;
      }
      .abstract {
          font-size: 16px;
          color: #666;
      }
      </style>
  """

  table_html = "<table>"
  for row in data:
      table_html += "<tr>"
      for i, item in enumerate(row):
          if i == 0:
              table_html += f"<td class='title'>{item}</td>"
          else:
              table_html += f"<td class='abstract'>{item}</td>"
      table_html += "</tr>"
  table_html += "</table>"

  display(HTML(table_style + table_html))

In [ ]:
# (candidate_titles, candidate_abstracts, candidate_scores), query = eval_model('specter', sentences, 'abstract', 'abstract', 'method', papers)
# (candidate_titles, candidate_abstracts, candidate_scores), query = eval_model('sentbert-pp', sentences, 'background_label', 'abstract', 'background', papers)
# (candidate_titles, candidate_abstracts, candidate_scores), query = eval_model('sentbert-pp', sentences, 'method_label', 'abstract', 'method', papers)
(candidate_titles, candidate_abstracts, candidate_scores), query = eval_model('sentbert-pp', sentences, 'result_label', 'abstract', 'result', papers)
displayResults(candidate_titles, candidate_abstracts, candidate_scores, query)

1198964 Edge-Linear First-Order Dependency Parsing with Undirected Minimum Spanning Tree Inference
10052042 Learning Topic-Sensitive Word Representations
11629674 Characterizing the Language of Online Communities and its Relation to Community Reception
1587 Get out the vote: Determining support or opposition from Congressional floor-debate transcripts
174799296 From Balustrades to Pierre Vinken: Looking for Syntax in Transformer Self-Attentions
1306065 A Convolutional Neural Network for Modelling Sentences
53080736 FewRel: A Large-Scale Supervised Few-Shot Relation Classification Dataset with State-of-the-Art Evaluation
2090262 Minimizing Manual Annotation Cost In Supervised Training From Corpora
2865563 Improving Statistical Machine Translation for a Resource-Poor Language Using Related Resource-Rich Languages
11844559 Unsupervised Induction of Semantic Roles within a Reconstruction-Error Minimization Framework
2468783 Similarity of Semantic Relations
5052952 Veracity Computing from L

Candidate Title,Candidate Abstract,Score
Multi-Granularity Self-Attention for Neural Machine Translation:,"Current state-of-the-art neural machine translation (NMT) uses a deep multi-head self-attention network with no explicit phrase information.However, prior work on statistical machine translation has shown that extending the basic translation unit from words to phrases has produced substantial improvements, suggesting the possibility of improving NMT performance from explicit modeling of phrases.In this work, we present multi-granularity self-attention (Mg-Sa): a neural network that combines multi-head self-attention and phrase modeling.Specifically, we train several attention heads to attend to phrases in either n-gram or syntactic formalism.Moreover, we exploit interactions among phrases to enhance the strength of structure modeling - a commonly-cited weakness of self-attention.Experimental results on WMT14 English-to-German and NIST Chinese-to-English translation tasks show the proposed approach consistently improves performance.Targeted linguistic analysis reveals that Mg-Sa indeed captures useful phrase information at various levels of granularities.",1
Phrase-level Self-Attention Networks for Universal Sentence Encoding:,"AbstractUniversal sentence encoding is a hot topic in recent NLP research.Attention mechanism has been an integral part in many sentence encoding models, allowing the models to capture context dependencies regardless of the distance between elements in the sequence.Fully attention-based models have recently attracted enormous interest due to their highly parallelizable computation and significantly less training time.However, the memory consumption of their models grows quadratically with sentence length, and the syntactic information is neglected.To this end, we propose Phrase-level Self-Attention Networks (PSAN) that perform self-attention across words inside a phrase to capture context dependencies at the phrase level, and use the gated memory updating mechanism to refine each word's representation hierarchically with longer-term context dependencies captured in a larger phrase.As a result, the memory consumption can be reduced because the self-attention is performed at the phrase level instead of the sentence level.At the same time, syntactic information can be easily integrated in the model.Experiment results show that PSAN can achieve the state-ofthe-art transfer performance across a plethora of NLP tasks including sentence classification, natural language inference and sentence textual similarity.",2
Analyzing the Structure of Attention in a Transformer Language Model:,"The Transformer is a fully attention-based alternative to recurrent networks that has achieved state-of-the-art results across a range of NLP tasks.In this paper, we analyze the structure of attention in a Transformer language model, the GPT-2 small pretrained model.We visualize attention for individual instances and analyze the interaction between attention and syntax over a large corpus.We find that attention targets different parts of speech at different layer depths within the model, and that attention aligns with dependency relations most strongly in the middle layers.We also find that the deepest layers of the model capture the most distant relationships.Finally, we extract exemplar sentences that reveal highly specific patterns targeted by particular attention heads.",2
Dependency-Aware Named Entity Recognition with Relative and Global Attentions:,"Named entity recognition is one of the core tasks in NLP.Although many improvements have been made on this task during the last years, the state-of-the-art systems do not explicitly take into account the recursive nature of language.Instead of only treating the text as a plain sequence of words, we incorporate a linguistically-inspired way to recognize entities based on syntax and tree structures.Our model exploits syntactic relationships among words using a Tree-LSTM guided 

In [ ]:
# (candidate_titles, candidate_abstracts, candidate_scores), query = eval_model('finetuned-specter', sentences, 'abstract', 'abstract', 'method', papers)
# (candidate_titles, candidate_abstracts, candidate_scores), query = eval_model('finetuned-sentbert-pp', sentences, 'background_label', 'abstract', 'background', papers)
# (candidate_titles, candidate_abstracts, candidate_scores), query = eval_model('finetuned-sentbert-pp', sentences, 'method_label', 'abstract', 'method', papers)
(candidate_titles, candidate_abstracts, candidate_scores), query = eval_model('finetuned-sentbert-pp', sentences, 'result_label', 'abstract', 'result', papers)
displayResults(candidate_titles, candidate_abstracts, candidate_scores, query)

1198964 Edge-Linear First-Order Dependency Parsing with Undirected Minimum Spanning Tree Inference
10052042 Learning Topic-Sensitive Word Representations
11629674 Characterizing the Language of Online Communities and its Relation to Community Reception
1587 Get out the vote: Determining support or opposition from Congressional floor-debate transcripts
174799296 From Balustrades to Pierre Vinken: Looking for Syntax in Transformer Self-Attentions
1306065 A Convolutional Neural Network for Modelling Sentences
53080736 FewRel: A Large-Scale Supervised Few-Shot Relation Classification Dataset with State-of-the-Art Evaluation
2090262 Minimizing Manual Annotation Cost In Supervised Training From Corpora
2865563 Improving Statistical Machine Translation for a Resource-Poor Language Using Related Resource-Rich Languages
11844559 Unsupervised Induction of Semantic Roles within a Reconstruction-Error Minimization Framework
2468783 Similarity of Semantic Relations
5052952 Veracity Computing from L

Candidate Title,Candidate Abstract,Score
Multi-Granularity Self-Attention for Neural Machine Translation:,"Current state-of-the-art neural machine translation (NMT) uses a deep multi-head self-attention network with no explicit phrase information.However, prior work on statistical machine translation has shown that extending the basic translation unit from words to phrases has produced substantial improvements, suggesting the possibility of improving NMT performance from explicit modeling of phrases.In this work, we present multi-granularity self-attention (Mg-Sa): a neural network that combines multi-head self-attention and phrase modeling.Specifically, we train several attention heads to attend to phrases in either n-gram or syntactic formalism.Moreover, we exploit interactions among phrases to enhance the strength of structure modeling - a commonly-cited weakness of self-attention.Experimental results on WMT14 English-to-German and NIST Chinese-to-English translation tasks show the proposed approach consistently improves performance.Targeted linguistic analysis reveals that Mg-Sa indeed captures useful phrase information at various levels of granularities.",1
Phrase-level Self-Attention Networks for Universal Sentence Encoding:,"AbstractUniversal sentence encoding is a hot topic in recent NLP research.Attention mechanism has been an integral part in many sentence encoding models, allowing the models to capture context dependencies regardless of the distance between elements in the sequence.Fully attention-based models have recently attracted enormous interest due to their highly parallelizable computation and significantly less training time.However, the memory consumption of their models grows quadratically with sentence length, and the syntactic information is neglected.To this end, we propose Phrase-level Self-Attention Networks (PSAN) that perform self-attention across words inside a phrase to capture context dependencies at the phrase level, and use the gated memory updating mechanism to refine each word's representation hierarchically with longer-term context dependencies captured in a larger phrase.As a result, the memory consumption can be reduced because the self-attention is performed at the phrase level instead of the sentence level.At the same time, syntactic information can be easily integrated in the model.Experiment results show that PSAN can achieve the state-ofthe-art transfer performance across a plethora of NLP tasks including sentence classification, natural language inference and sentence textual similarity.",2
Analyzing the Structure of Attention in a Transformer Language Model:,"The Transformer is a fully attention-based alternative to recurrent networks that has achieved state-of-the-art results across a range of NLP tasks.In this paper, we analyze the structure of attention in a Transformer language model, the GPT-2 small pretrained model.We visualize attention for individual instances and analyze the interaction between attention and syntax over a large corpus.We find that attention targets different parts of speech at different layer depths within the model, and that attention aligns with dependency relations most strongly in the middle layers.We also find that the deepest layers of the model capture the most distant relationships.Finally, we extract exemplar sentences that reveal highly specific patterns targeted by particular attention heads.",2
Dependency-Aware Named Entity Recognition with Relative and Global Attentions:,"Named entity recognition is one of the core tasks in NLP.Although many improvements have been made on this task during the last years, the state-of-the-art systems do not explicitly take into account the recursive nature of language.Instead of only treating the text as a plain sequence of words, we incorporate a linguistically-inspired way to recognize entities based on syntax and tree structures.Our model exploits syntactic relationships among words using a Tree-LSTM guided 